In [12]:
import yfinance as yf
import json
import numpy as np
import os
import pandas as pd
from stock_file_creation import SYMBOLS

In [14]:
# Dictionary to collect data to create a DF later
data = {
    'Symbol': [],
    'Name': [],
    'Industry': [],
    'EPS (fwd)': [],
    'P/E (fwd)': [],
    'PEG': [],
    'FCFY' : [],
    'PB': [],
    'ROE' : [],
    'P/S (trail)': [],
    'DPR' : [],
    'DY' : [],
    'CR' : [],
    'Beta': [],
    'Price': [],
    '52w Low': [],
    '52w High': []
    }

#function to append data to dictionary
def load_data(json_data):
    data['Symbol'].append(json_data.get('symbol'))
    data['Name'].append(json_data.get('longName'))
    data['Industry'].append(json_data.get('industry'))
    data['Price'].append(json_data.get('currentPrice'))

    # Could be that some indicators are not available; use NaN if this is the case
    
    if 'forwardEps' in json_data:
        data['EPS (fwd)'].append(json_data.get('forwardEps'))
    else:
        data['EPS (fwd)'].append(np.nan)
        
    if 'forwardPE' in json_data:
        data['P/E (fwd)'].append(json_data.get('forwardPE'))
    else:
        data['P/E (fwd)'].append(np.nan)
        
    if 'pegRatio' in json_data:
        data['PEG'].append(json_data.get('pegRatio'))
    else:
        data['PEG'].append(np.nan)

    if ('freeCashflow' in json_data and json_data.get('freeCashflow') is not None) and ('marketCap' in json_data and json_data.get('marketCap') is not None):
        fcfy = (json_data.get('freeCashflow') / json_data.get('marketCap')) * 100
        data['FCFY'].append(round(fcfy, 2))
    else:
        data['FCFY'].append(np.nan)


    if 'priceToBook' in json_data:
        data['PB'].append(json_data.get('priceToBook'))
    else:
        data['PB'].append(np.nan)

    if 'returnOnEquity' in json_data:
        data['ROE'].append(json_data.get('returnOnEquity'))
    else:
        data['ROE'].append(np.nan)
        
    if 'priceToSalesTrailing12Months' in json_data:
        data['P/S (trail)'].append(json_data.get('priceToSalesTrailing12Months'))
    else:
        data['P/S (trail)'].append(np.nan)

    if json_data.get('payoutRatio') is not None:
        data['DPR'].append(json_data.get('payoutRatio') * 100)
    else:
        data['DPR'].append(np.nan)

    if 'dividendYield' in json_data:
        data['DY'].append(json_data.get('dividendYield'))
    else:
        data['DY'].append(0.0)

    if 'beta' in json_data:
        data['Beta'].append(json_data.get('beta'))
    else:
        data['Beta'].append(np.nan)

    if 'currentRatio' in json_data:
        data['CR'].append(json_data.get('currentRatio'))
    else:
        data['CR'].append(np.nan)

    if 'fiftyTwoWeekLow' in json_data:
        data['52w Low'].append(json_data.get('fiftyTwoWeekLow'))
    else:
        data['52w Low'].append(np.nan)
        
    if 'fiftyTwoWeekHigh' in json_data:    
        data['52w High'].append(json_data.get('fiftyTwoWeekHigh'))
    else:
        data['52w High'].append(np.nan)

for symbol in SYMBOLS:
    # Specify the full path to load JSON data
    file_name = f'C:/D/Other/Data Engineering/yfinance/stockout/{symbol}.json'    
    
    try:
        # Open the file in read mode
        with open(file_name, 'r') as file:
            # Use json.load() to parse the JSON data from the file
            load_data(json.load(file))
    except FileNotFoundError:
        print(f"File '{file_name}' not found.")
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON data: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")


In [4]:
df = pd.DataFrame(data)

# Save any stocks with NaN values
df_exceptions = df[df.isna().any(axis=1)]

# Remove any stocks with NaN values
df=df.dropna()

# Reset index after dropping rows with NaN values
df.reset_index(drop=True, inplace=True)

# Add 52 week price range
df['52w Range'] = ((df['Price'] - df['52w Low'])/(df['52w High'] - df['52w Low']))*100

df_exceptions

,Symbol,Name,Industry,EPS (fwd),P/E (fwd),PEG,FCFY,PB,ROE,P/S (trail),DPR,DY,CR,Beta,Price,52w Low,52w High
0,APPL,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,NaN,NaN,NaN,NaN
10,JPM,JPMorgan Chase & Co.,Banks - Diversified,15.75,10.937778,3.45,NaN,1.717547,0.16932,3.496671,24.159999,0.0244,NaN,1.111,172.27,123.11,173.38
20,ADBE,Adobe Inc.,Software - Infrastructure,20.36,27.730843,2.20,NaN,15.552432,0.35513,13.244494,NaN,0.0000,1.343,1.330,564.60,318.60,633.89
22,CRM,"Salesforce, Inc.",Software - Application,9.56,26.267780,1.15,4.74,4.188963,0.04414,7.159220,NaN,0.0000,1.042,1.281,251.12,142.59,268.36


In [5]:
df

,Symbol,Name,Industry,EPS (fwd),P/E (fwd),PEG,FCFY,PB,ROE,P/S (trail),DPR,DY,CR,Beta,Price,52w Low,52w High,52w Range
0,MSFT,Microsoft Corporation,Software - Infrastructure,12.96,28.375770,2.12,1.87,12.381321,0.39107,12.519875,26.360000,0.0082,1.663,0.876,367.75,226.41,384.30,89.518019
1,AMZN,"Amazon.com, Inc.",Internet Retail,3.61,40.232690,0.62,2.42,8.199628,0.12531,2.709087,0.000000,0.0000,0.985,1.163,145.24,87.08,155.63,84.843180
2,NVDA,NVIDIA Corporation,Semiconductors,20.44,24.020058,0.39,1.16,36.397804,0.69173,27.026876,2.110000,0.0003,3.588,1.642,490.97,151.41,505.48,95.901940
3,GOOGL,Alphabet Inc.,Internet Content & Information,6.69,20.288490,1.21,4.14,6.230434,0.25334,5.749289,0.000000,0.0000,2.043,1.054,135.73,85.83,142.68,87.774846
4,TSLA,"Tesla, Inc.",Auto Manufacturers,3.78,62.828045,94.24,0.21,14.121180,0.22460,7.870415,0.000000,0.0000,1.690,2.316,237.49,114.92,299.29,66.480447
5,META,"Meta Platforms, Inc.",Internet Content & Information,17.54,20.065563,0.76,3.25,6.333340,0.22275,7.124218,0.000000,0.0000,2.567,1.221,351.95,127.15,361.90,95.761448
6,UNH,UnitedHealth Group Incorporated,Healthcare Plans,27.89,19.267840,1.69,4.32,5.882971,0.25299,1.380725,30.640000,0.0140,0.773,0.564,537.38,445.68,554.70,84.113007
7,XOM,Exxon Mobil Corporation,Oil & Gas Integrated,9.17,11.191930,-1.04,5.04,2.036592,0.21168,1.170659,36.150002,0.0370,1.421,0.960,102.63,97.48,120.70,22.179156
8,AVGO,Broadcom Inc.,Semiconductors,55.42,18.933239,1.63,2.74,17.718039,0.60312,13.713699,55.790000,0.0200,2.815,1.241,1049.28,549.99,1151.82,82.961966
9,LLY,Eli Lilly and Company,Drug Manufacturers - General,12.38,49.963650,3.24,0.66,49.575220,0.46461,18.308329,79.170000,0.0084,1.046,0.307,618.55,309.20,636.41,94.541732


In [6]:
def make_pretty(styler):
    # Column formatting
    styler.format({'EPS (fwd)': '${:.2f}', 'P/E (fwd)': '{:.2f}', 'PEG': '{:.2f}',
                   'FCFY': '{:.2f}%', 'PB' : '{:.2f}', 'ROE' : '{:.2f}', 'P/S (trail)': '{:.2f}',
                   'DPR': '{:.2f}%', 'DY': '{:.2f}%', 'CR' : '{:.2f}', 'Beta': '{:.2f}', '52w Low': '${:.2f}',
                   'Price': '${:.2f}', '52w High': '${:.2f}', '52w Range': '{:.2f}%'
                  })
    # Set the bar visualization
    styler.bar(subset = ['52w Range'], align = "mid", color = ["salmon", "cornflowerblue"])

    # Grid
    styler.set_properties(**{'border': '0.1px solid black'})

    # Set background gradients
    styler.background_gradient(subset=['EPS (fwd)'], cmap='Greens')
    styler.background_gradient(subset=['P/E (fwd)'], cmap='Greens')
    styler.background_gradient(subset=['PEG'], cmap='Greens')
    styler.background_gradient(subset=['FCFY'], cmap='Greens')
    styler.background_gradient(subset=['PB'], cmap='Greens')
    styler.background_gradient(subset=['ROE'], cmap='Greens')
    styler.background_gradient(subset=['P/S (trail)'], cmap='Greens')
    styler.background_gradient(subset=['DPR'], cmap='Greens')
    styler.background_gradient(subset=['DY'], cmap='Greens')
    styler.background_gradient(subset=['CR'], cmap='Greens')

    # No index
    styler.hide(axis='index')

    # Tooltips
    styler.set_tooltips(
        ttips, css_class='tt-add',
        props=[
            ('visibility', 'hidden'),
            ('position', 'absolute'),
            ('background-color', 'salmon'),
            ('color', 'black'),
            ('z-index', 1),
            ('padding', '3px 3px'),
            ('margin', '2px')
        ]
    )
    # Left text alignment for some columns
    styler.set_properties(subset=['Symbol', 'Name', 'Industry'], **{'text-align': 'left'})
    return styler

In [7]:
def populate_tt(df, tt_data, col_name):
    stats = df[col_name].describe()
    
    per25 = round(stats.loc['25%'], 2)
    per50 = round(stats.loc['50%'], 2)
    per75 = round(stats.loc['75%'], 2)

    # Get position based on the column name
    pos = df.columns.to_list().index(col_name)
    
    for index, row in df.iterrows():
        pe = row[col_name]
        if pe == stats.loc['min']:
            tt_data[index][pos] = 'Lowest'
        elif pe == stats.loc['max']:
            tt_data[index][pos] = 'Hightest'
        elif pe <= per25:
            tt_data[index][pos] = '25% of companies under {}'.format(per25)
        elif pe <= per50:
            tt_data[index][pos] = '50% of companies under {}'.format(per50)
        elif pe <= per75:
            tt_data[index][pos] = '75% of companies under {}'.format(per75)
        else:
            tt_data[index][pos] = '25% of companies over {}'.format(per75)  

In [8]:
# Initialize tool tip data - each column is set to '' for each row
tt_data = [['' for x in range(len(df.columns))] for y in range(len(df))]

# Gather tool tip data for indicators
populate_tt(df, tt_data, 'EPS (fwd)')
populate_tt(df, tt_data, 'P/E (fwd)')
populate_tt(df, tt_data, 'PEG')
populate_tt(df, tt_data, 'FCFY')
populate_tt(df, tt_data, 'PB')
populate_tt(df, tt_data, 'ROE')
populate_tt(df, tt_data, 'P/S (trail)')
populate_tt(df, tt_data, 'DPR')
populate_tt(df, tt_data, 'DY')
populate_tt(df, tt_data, 'CR')

# Create a tool tip DF
ttips = pd.DataFrame(data=tt_data, columns=df.columns, index=df.index)

# Add table caption and styles to DF
df.style.pipe(make_pretty).set_caption('Fundamental Indicators').set_table_styles(
    [{'selector': 'th.col_heading', 'props': 'text-align: center'},
     {'selector': 'caption', 'props': [('text-align', 'center'),
                                       ('font-size', '11pt'), ('font-weight', 'bold')]}])

Symbol,Name,Industry,EPS (fwd),P/E (fwd),PEG,FCFY,PB,ROE,P/S (trail),DPR,DY,CR,Beta,Price,52w Low,52w High,52w Range
MSFT,Microsoft Corporation,Software - Infrastructure,$12.96,28.38,2.12,1.87%,12.38,0.39,12.52,26.36%,0.01%,1.66,0.88,$367.75,$226.41,$384.30,89.52%
AMZN,"Amazon.com, Inc.",Internet Retail,$3.61,40.23,0.62,2.42%,8.20,0.13,2.71,0.00%,0.00%,0.98,1.16,$145.24,$87.08,$155.63,84.84%
NVDA,NVIDIA Corporation,Semiconductors,$20.44,24.02,0.39,1.16%,36.40,0.69,27.03,2.11%,0.00%,3.59,1.64,$490.97,$151.41,$505.48,95.90%
GOOGL,Alphabet Inc.,Internet Content & Information,$6.69,20.29,1.21,4.14%,6.23,0.25,5.75,0.00%,0.00%,2.04,1.05,$135.73,$85.83,$142.68,87.77%
TSLA,"Tesla, Inc.",Auto Manufacturers,$3.78,62.83,94.24,0.21%,14.12,0.22,7.87,0.00%,0.00%,1.69,2.32,$237.49,$114.92,$299.29,66.48%
META,"Meta Platforms, Inc.",Internet Content & Information,$17.54,20.07,0.76,3.25%,6.33,0.22,7.12,0.00%,0.00%,2.57,1.22,$351.95,$127.15,$361.90,95.76%
UNH,UnitedHealth Group Incorporated,Healthcare Plans,$27.89,19.27,1.69,4.32%,5.88,0.25,1.38,30.64%,0.01%,0.77,0.56,$537.38,$445.68,$554.70,84.11%
XOM,Exxon Mobil Corporation,Oil & Gas Integrated,$9.17,11.19,-1.04,5.04%,2.04,0.21,1.17,36.15%,0.04%,1.42,0.96,$102.63,$97.48,$120.70,22.18%
AVGO,Broadcom Inc.,Semiconductors,$55.42,18.93,1.63,2.74%,17.72,0.60,13.71,55.79%,0.02%,2.81,1.24,$1049.28,$549.99,$1151.82,82.96%
LLY,Eli Lilly and Company,Drug Manufacturers - General,$12.38,49.96,3.24,0.66%,49.58,0.46,18.31,79.17%,0.01%,1.05,0.31,$618.55,$309.20,$636.41,94.54%
